# Import Libraries

In [1]:
import os

os.chdir('..')
print(os.getcwd())

/Users/ani/Projects/6_stock_portfolio_recommendation


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas_gbq

import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from utils.data_ingestion import *
from src.parameters import *

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

from tqdm import tqdm
import yfinance as yf
import lxml

# Get List of Tickers

In [ ]:
df_sp500 = get_base_data('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_sp500[:5]

# Create Dataset with Features

In [ ]:
def calculate_annualized_returns(ticker_symbol, df_input, period="5y"):
    """Calculate annualized returns for a given ticker over a specified period.
    Args:
        ticker (yfinance.Ticker): The ticker object for the stock.
        period (str): The period over which to calculate returns (default is "5y").
    Returns:
        pd.Series: A series of annualized returns.
    """

    try:
        ticker = yf.Ticker(ticker_symbol)

        # Get historical market data
        hist = ticker.history(period=period).reset_index()
        hist['Date'] = pd.to_datetime(hist['Date'])
        hist['Year'] = hist['Date'].dt.year
        current_year = hist['Year'].max()    

        
        # average annualized return
        annualized_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) ** (1 / (current_year - hist['Year'].min())) - 1) * 100
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Annualized_Return'] = np.round(annualized_return, 2)

        # Calculate Percent Returns
        for year in hist['Year'].sort_values(ascending=False).unique():
            if year == current_year:
                ytd_return = ((hist[hist['Year'] == year]['Close'].iloc[-1] - hist[hist['Year'] == year]['Close'].iloc[0]) / hist[hist['Year'] == year]['Close'].iloc[0]) * 100
                df_input.loc[df_input['Ticker'] == ticker_symbol, 'YTD_Pct_Return'] = np.round(ytd_return, 2)
            elif year < current_year:
                annual_return = ((hist[hist['Year'] == year]['Close'].iloc[-1] - hist[hist['Year'] == year]['Close'].iloc[0]) / hist[hist['Year'] == year]['Close'].iloc[0]) * 100
                df_input.loc[df_input['Ticker'] == ticker_symbol, f'{year}_Pct_Return'] = np.round(annual_return, 2)
        
        # Get market cap
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Market_Cap'] = ticker.info.get('marketCap', np.nan)
        
        # 24 Hour Change
        hist['24_Hour_Change'] = hist['Close'].pct_change(periods=1) * 100
        df_input.loc[df_input['Ticker'] == ticker_symbol, '24_Hour_Change'] = np.round(hist['24_Hour_Change'].iloc[-1], 2)

        # 7 day Change
        hist['7_Day_Change'] = hist['Close'].pct_change(periods=7) * 100
        df_input.loc[df_input['Ticker'] == ticker_symbol, '7_Day_Change'] = np.round(hist['7_Day_Change'].iloc[-1], 2)

        # 30 Day Change
        hist['30_Day_Change'] = hist['Close'].pct_change(periods=30) * 100
        df_input.loc[df_input['Ticker'] == ticker_symbol, '30_Day_Change'] = np.round(hist['30_Day_Change'].iloc[-1], 2)
        
        # Calculate 200 Day Moving Average & Pct Difference from it
        hist['200_MA'] = hist['Close'].rolling(window=200).mean()
        hist['Pct_Diff_200_MA'] = ((hist['Close'] - hist['200_MA']) / hist['200_MA']) * 100
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Pct_Diff_200_MA'] = np.round(hist['Pct_Diff_200_MA'].iloc[-1], 2)
        
        # Calculate Volatility
        hist['Daily_Return'] = hist['Close'].pct_change()
        mean_daily_return = hist['Daily_Return'].mean()
        volatility = (((hist['Daily_Return'] - mean_daily_return) ** 2).mean() ** 0.5) * np.sqrt(252)  # Annualize the volatility
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Annualized_Volatility'] = np.round(volatility, 2)

        # Calculate Sharpe Ratio
        risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
        sharpe_ratio = (annualized_return - risk_free_rate) / volatility
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Sharpe_Ratio'] = np.round(sharpe_ratio, 2)

        # Calculate Beta
        benchmark_ticker = '^GSPC'  # S&P 500 as benchmark
        benchmark = yf.Ticker(benchmark_ticker)
        benchmark_hist = benchmark.history(period=period).reset_index()
        benchmark_hist['Date'] = pd.to_datetime(benchmark_hist['Date'])
        benchmark_hist['Daily_Return'] = benchmark_hist['Close'].pct_change()
        hist = hist.merge(benchmark_hist[['Date', 'Daily_Return']], on='Date', suffixes=('', '_Benchmark'))
        covariance = hist['Daily_Return'].cov(hist['Daily_Return_Benchmark'])
        benchmark_variance = hist['Daily_Return_Benchmark'].var()
        beta = covariance / benchmark_variance if benchmark_variance != 0 else np.nan
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Beta'] = np.round(beta, 2)

        # Years since founded
        df_input.loc[df_input['Ticker'] == ticker_symbol, 'Years_Since_Founded'] = current_year - int(df_input[df_input['Ticker'] == ticker_symbol]['Founded'].max()[:4])

        return df_input
    
    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")
        return df_input



In [ ]:
for tickers in tqdm(df_sp500['Ticker'].unique().tolist()):
    df_sp500 = calculate_annualized_returns(ticker_symbol=tickers, df_input=df_sp500, period="5y")

df_sp500 = df_sp500.sort_values('Market_Cap', ascending=False).reset_index(drop=True)
df_sp500

# Save to BigQuery

In [ ]:
# Insert values in a table
save_table_to_bigquery(df=df_sp500, dataset_id=dataset_id, table_id=table_id)

# Load from BigQuery

In [4]:
df_sp500 = load_table_from_bigquery(dataset_id=dataset_id, table_id=table_id, project_id=PROJECT_ID)
df_sp500

Downloading: 100%|██████████|


,Ticker,Company_Name,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Pct_From_All_Time_High,24_Hour_Change,...,2022_Pct_Return,2021_Pct_Return,2020_Pct_Return,Market_Cap,Pct_Diff_200_MA,Annualized_Volatility,Sharpe_Ratio,Beta,Years_Since_Founded,Update_Date
0,OMC,Omnicom Group,Communication Services,Advertising,"New York City, New York",1997-12-31,29989,1986,-31.06,0.17,...,14.75,24.86,11.00,1.401192e+10,-18.50,0.28,28.30,0.81,39.0,2025-06-09
1,IPG,Interpublic Group of Companies (The),Communication Services,Advertising,"New York City, New York",1992-10-01,51644,1961 (1930),-38.37,0.09,...,-8.31,67.40,29.25,8.533484e+09,-15.56,0.30,28.21,0.98,64.0,2025-06-09
2,WBD,Warner Bros. Discovery,Communication Services,Broadcasting,"New York City, New York",2022-04-11,1437107,2022 (Warner Bros. 1923),-87.67,-2.95,...,-62.57,-23.15,29.87,2.357789e+10,1.79,0.54,-30.07,1.26,3.0,2025-06-09
3,FOX,Fox Corporation (Class B),Communication Services,Broadcasting,"New York City, New York",2019-03-19,1754301,2019,-8.14,-0.46,...,-17.53,22.18,-2.05,2.325886e+10,10.04,0.29,42.77,0.75,6.0,2025-06-09
4,FOXA,Fox Corporation (Class A),Communication Services,Broadcasting,"New York City, New York",2019-03-19,1754301,2019,-6.27,-0.50,...,-18.25,29.62,-2.58,2.325072e+10,11.98,0.30,46.52,0.72,6.0,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,CNP,CenterPoint Energy,Utilities,Multi-Utilities,"Houston, Texas",1985-07-31,1130310,1882,-5.86,-0.19,...,11.16,34.27,19.26,2.389639e+10,12.79,0.22,79.50,0.49,143.0,2025-06-09
499,CMS,CMS Energy,Utilities,Multi-Utilities,"Jackson, Michigan",1957-03-04,811156,1886,-7.05,-0.49,...,0.70,12.29,3.18,2.093868e+10,1.09,0.20,31.64,0.33,139.0,2025-06-09
500,NI,NiSource,Utilities,Multi-Utilities,"Merrillville, Indiana",2000-11-02,1111711,1912,-3.88,0.18,...,3.00,26.74,-4.35,1.852216e+10,7.28,0.21,65.19,0.47,113.0,2025-06-09
501,PNW,Pinnacle West Capital,Utilities,Multi-Utilities,"Phoenix, Arizona",1999-10-04,764622,1985,-5.41,0.36,...,14.30,-6.77,4.35,1.067547e+10,1.33,0.22,33.60,0.40,40.0,2025-06-09
